# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# options.add_argument('headless') # 창 숨기기


In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (X11; Ubuntu; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.114 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36
Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.101 Safari/537.36


In [3]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


C:\Users\user\AppData\Local\Temp\ipykernel_15528\1335855278.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


In [5]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_6월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11727, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/yeojuhangul/222017304299,여주 힐링 여행지 2곳 소개합니다(여주 여행코스),2020.06.30.
1,https://blog.naver.com/sy990520/222017303058,대중가요는 시대를 담았다 [국립한글박물관 기자단] 기획특별전...,2020.06.30.
2,https://blog.naver.com/guryesns/222017303062,구례 백두대간 생태교육장 체험 프로그램 누려~~,2020.06.30.
3,https://blog.naver.com/shirayuki_111/222017303214,신비아파트 더블X: 6개의 예언 귀신 정리,2020.06.30.
4,https://blog.naver.com/jsshopholic/222017301970,인사동 브런치 한식 레스토랑 @ 스페이스오,2020.06.30.
...,...,...,...
11722,https://blog.naver.com/dlagustnkh/221986773931,[후불제상조] 올바른상조 믿을 수 있는 후불제상조 함께 하세요,2020.06.01.
11723,https://blog.naver.com/aa13245/221986771937,2020년 6월 1일 아웃풋,2020.06.01.
11724,https://blog.naver.com/saidsong/221986773077,생명연습 - 문학동네 한국문학전집 001,2020.06.01.
11725,https://blog.naver.com/lcwlimkong/221986770764,"힐링을 주는 녹지공간, 숲세권의 가치 증가",2020.06.01.


In [6]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11727


In [7]:
df = pd.DataFrame(columns = {'url','본문'})
df.to_csv("./나무문화/2020/나무문화_본문_2020_6월.csv", index=False, encoding='UTF-8')
df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')

In [8]:
len(df)

0

In [9]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_6월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_6월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    if blog_links.index(i) != df['본문'].tolist().index(a):
        break


  0%|          | 0/11727 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_15528\3763219390.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 

c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://tour.yd.go.kr/kor/part/guidebook.aspx?MC=603008
영덕문화관광
관광%20안내지도%20홈%20%3e%20참여마당%20%3e%20여행도우미%20%3e%20관광안내지도%20영덕여행을%20위한%20관광안내지도%20를%20서비스%20하고%20있습니다.
tour.yd.go.kr
삼사해상공원
동해의%20맑은%20정기가%20서린%20삼사해상공원은%20청정바다를%20한%20눈에%20바라볼수%20있을뿐%20아니라%20주위의%20경관이%20아름다워%20주말은%20물론%20평일에도%20가족단위%20행락인들이%20줄을%20잇는다.%20이북%205도민의%20망향의%20설움을%20달래기%20위해%2095년도에%20세워진망향탑과%20경북개도%20100주년%20기념사업인%20경북대종,%20공연장과%20폭포%20기타%20편의시설이%20설치되어%20있다.%20이곳에서%201997년%201월%201일%20처음%20개최한%20%22해맞이축제%22는%20신년의%20소망을%20기원하는%20사람들로%20대성황을%20이루었고,%20이%20행사는%20매년%20열리고%20있으며%20이어서%20또%20하나의%20문화축제로%20자리매김%20되고%20있다.
5월부터%2010월까지%20매주%20토요일%20오후에는%20지역음악동호회가%20참여하는%20관광명소%20주말공연이%20열려%20삼사해상공원을%20찾는%20관광객에게%20즐거움을%20더해주고%20있다.
위치%20:%20강구면%20삼사리
' with link or location/anchor > 2079 characters since it exceeds Excel's limit for URLS
  warn("Ignoring URL '%s' with link or location/anchor > %d "


993 993
994 994
995 995
996 996
997 997
998 998
999 999


...

c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://blog.naver.com/gykim67/221823192026
문화가%20있는%20날%20-%20슬로시티%20예산%20대흥(1)
'느리고%20한갓지게,%20삶이%20살찌는%20슬로시티%20예산%20대흥'이라고%20슬로시티%20예산%20대흥을%20소개합니...
blog.naver.com
2019년%209월%2027일%20충남%20예산%20슬로시티%20대흥%20방문,%20그%20두번째%20이야기입니다.
솟대와%20달걀꾸러미%20만들기%20체험을%20마치고,%20마을%20탐방에%20나섰습니다.
의좋은형제공원까지만%20갔었기%20때문에%20마을%20안쪽은%20가보질%20않아서%20궁금하긴%20했습니다.
50m
©%20NAVER%20Corp.
의좋은형제마을
충청남도%20예산군%20대흥면%20의좋은형제길%2051
동헌이%20있을%20정도로%20제법%20큰%20마을이었기에%20아직도%20우체국과%20보건지소,%20행정복지센터가%20마을%20안쪽에%20자리하고%20있었습니다.
대개의%20경우%20큰길가에%20있을%20법한데%20말이죠.
대흥초등학교가%20개교%20100년이%20넘었다고%20하네요.
예산에서도%20큰%20마을이었기에%20동헌이%20있었겠지요.
'의좋은%20형제'는%20밤새%20상대의%20창고로%20볏단을%20나르다가%20우연히%20만난%20형제이야기로%20초등학교,%20아니%20국민학교%20다닐%20때%20국어교과서에%20실린%20내용입니다.
그%20이야기를%20라면%20CF에서도%20사용했었죠.%20형님%20먼저,%20아우%20먼저,%20그럼%20내가%20먼저...
콤비라%20불리던%20코미디언%20두%20분이%20출연했던%20광고.%20지금도%20기억합니다.
대흥마을에서%20이성만형제%20효제비가%20발견되며%20실화로%20밝혀졌다고%20해요.
효제비와%20이야기가%

1047 1047
1048 1048
1049 1049
1050 1050
1051 1051
1052 1052
1053 1053
1054 1054
1055 1055
1056 1056
1057 1057
1058 1058
1059 1059
1060 1060
1061 1061
1062 1062
1063 1063
1064 1064
1065 1065
1066 1066
1067 1067
1068 1068
1069 1069
1070 1070
1071 1071
1072 1072
1073 1073
1074 1074
1075 1075
1076 1076
1077 1077
1078 1078
1079 1079
1080 1080
1081 1081
1082 1082
1083 1083
1084 1084
1085 1085
1086 1086
1087 1087
1088 1088
1089 1089
1090 1090
1091 1091
1092 1092
1093 1093
1094 1094
1095 1095
1096 1096
1097 1097
1098 1098
1099 1099
1100 1100
1101 1101
1102 1102
1103 1103
1104 1104
1105 1105
1106 1106
1107 1107
1108 1108
1109 1109
1110 1110
1111 1111
1112 1112
1113 1113
1114 1114
1115 1115
1116 1116
1117 1117
1118 1118
1119 1119
1120 1120
1121 1121
1122 1122
1123 1123
1124 1124
1125 1125
1126 1126
1127 1127
1128 1128
1129 1129
1130 1130
1131 1131
1132 1132
1133 1133
1134 1134
1135 1135
1136 1136
1137 1137
1138 1138
1139 1139
1140 1140
1141 1141
1142 1142
1143 1143
1144 1144
1145 1145
1146 1146


c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://kin.naver.com/qna/detail.nhn?d1id=11&dirId=1116&docId=360831661&qb=7KeE7ZmU66Gg&enc=utf8&section=kin&rank=6&search_sort=3&spq=0
진화론%20관련%20질문
생명과학에%20관심이%20있는%20한%20학생입니다.%20평소에%20궁금한%20것들이%20많아,%20이를%20해결해%20주셨으면%20합니다.%20좀%20장황한%20점%20양해%20부탁%20드립니다.%201.%20학교%20선생님께서는%20생명체의%20탄생이%20본래%20...
kin.naver.com
참솔
1.%20%5b그것을%20어떻게,%20왜%20구성하게%20된%20것인가요?%5d
%5b어떻게%5d는%20과학에서%20말할%20수%20있지만%20%5b왜%5d는%20말할%20수%20없습니다.%20이것은%20%5b산소%20하나와%20수소%20둘이%20어떻게,%20왜%20물을%20만드는%20것인가요?%5d와%20같은%20이야기죠.%20%5b어떻게%5d는%20%5b산소와%20수소의%20공유결합에%20의해%5d라%20말할%20수%20있지만%20%5b왜%5d는%20과학에서%20말할%20수%20없습니다.%20%5b왜%5d는%20과학이%20아니라%20종교에%20물어야%20할%20이야기죠.
%5b어떻게%5d를%20이야기해본다면,%20화학반응에%20의해%20복잡한%20유기물이%20합성될%20수%20있습니다.%20그%20복잡한%20유기물%20중에%20%5b자기복제분자%5d가%20생긴다면%20그%20자기복제분자(이를테면%20RNA%20등)가%20스스로를%20복제하며%20생명체로%20%5b화학진화%5d되었다는%20것이%20현재로서는%20정설입니다.%20믈론%20이것은%20아주%20간단하게%20설명한%20것이고%20자세한%20것은%20책을%20읽어

2027 2027
2028 2028
2029 2029
2030 2030
2031 2031
2032 2032
2033 2033
2034 2034
2035 2035
2036 2036
2037 2037
2038 2038
2039 2039
2040 2040
2041 2041
2042 2042
2043 2043
2044 2044
2045 2045
2046 2046
2047 2047
2048 2048
2049 2049
2050 2050
2051 2051
2052 2052
2053 2053
2054 2054
2055 2055
2056 2056
2057 2057
2058 2058
2059 2059
2060 2060
2061 2061
2062 2062
2063 2063
2064 2064
2065 2065
2066 2066
2067 2067
2068 2068
2069 2069
2070 2070
2071 2071
2072 2072
2073 2073
2074 2074
2075 2075
2076 2076
2077 2077
2078 2078
2079 2079
2080 2080
2081 2081
2082 2082
2083 2083
2084 2084
2085 2085
2086 2086
2087 2087
2088 2088
2089 2089
2090 2090
2091 2091
2092 2092
2093 2093
2094 2094
2095 2095
2096 2096
2097 2097
2098 2098
2099 2099
2100 2100
2101 2101
2102 2102
2103 2103
2104 2104
2105 2105
2106 2106
2107 2107
2108 2108
2109 2109
2110 2110
2111 2111
2112 2112
2113 2113
2114 2114
2115 2115
2116 2116
2117 2117
2118 2118
2119 2119


KeyboardInterrupt: 

In [17]:
df

,url,본문
0,https://blog.naver.com/jinidag/222104033228,"추석연휴 가볼만한곳 야외 나들이\n""이글은 한국민속촌으로부터 금전적 지원을 받아 작..."
1,https://blog.naver.com/taya2765/222104032204,복천박물관\n부산 동래구 복천로 63\n매일 09:00-18:00 (매주 마지막주 ...
2,https://blog.naver.com/yeonjae1213/222104028784,오늘 아침에 일어나서 엄마랑 극적 화해하고 기분 좋았다. 그런데 기상은 늦었다 10...
3,https://blog.naver.com/sunnybridge_/222104026647,호주 워킹홀리데이 상반기 시즌은 사진이 많이 없어서\n포스팅 할게 많이 없다 ㅠㅠ\...
4,https://blog.naver.com/lby56/222104022581,블로그\n공지 목록\n공지글\n글 제목 작성일\n(12)\n공지 이병렬 저서 출간 ...
5,https://blog.naver.com/qkrtiger/222104017905,컨디션 회복을 위한(?)\n걷기 ...\n아점 후\n형산강변 어싱 ㄱㄱ\n유강방면으...
6,https://blog.naver.com/tnqls11004/222104016240,안녕하세요. 강서구 SNS 서포터즈 '새로미 프렌즈'의 뚜비입니다! 오늘은 강서구에...
7,https://blog.naver.com/019088/222104016283,서로이웃 추가\n이번 달에 새롭게 추가되신 분들은 총 15분입니다. 블로그 운영하는...
8,https://blog.naver.com/jjggrr202/222104015943,blog\n일본 성지순례기(2018) 16개의 글\n일본 성지순례기(2018)\n목...


In [9]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))


214
28
0
